In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
os.environ['NEO4J_URI'] = os.getenv("NEO4J_URI")
os.environ['NEO4J_USERNAME'] = os.getenv("NEO4J_USERNAME")
os.environ['NEO4J_PASSWORD'] = os.getenv("NEO4J_PASSWORD")
groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(refresh_schema=False)
graph

In [19]:
movie_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie {id:toInteger(row.MovieId)})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREAH (actor in split(row.actors, '|') |
    MERCGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [23]:
movie_query

"\nLOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie {id:toInteger(row.MovieId)})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREAH (actor in split(row.actors, '|') |\n    MERCGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') |\n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n"

In [ ]:
graph.refresh_schema()
print(graph.schema)

In [ ]:
graph.query(movie_query)

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True)
chain

In [ ]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response